# Assignment 1, 8-Puzzle Programming

### Your homework must be implemented in this Notebook file. 
### You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
### You need to implement the three (four for grads) searching functions and the print result functions.
### For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.

In [14]:
###
#IMPORT PUZZLE CASES
###

puzzleCases = []
with open('Input8PuzzleCases.txt', 'r') as f:
    for rawLine in f:
        charLine = rawLine.strip('\n').split(',') #tokenize by commas, purge newlines
        intLine = [int(char) for char in charLine] #convert list members from strings to ints via list comprehension
        puzzleCases.append(intLine) #add each initial puzzle state to list as a tuple
f.close()

#global variable goalState
goalState = (0,1,2,3,4,5,6,7,8)

In [15]:
###
#GENERAL HELPER FUNCTIONS & IMPORTS
###
import time #for time metric

#figures out the next possible actions for a given state
#returns a dictionary mapping each action to a boolean saying whether or not the move is available at that state
def identifyActionsForState(state):
    index = findIndexOfZeroInState(state)
    #dictionary of actions available based on the position of the empty tile ("zero" tile)
    possibleActionsForZeroPosition = {
        0: {"Up": False, "Right": True, "Down": True, "Left": False},
        1: {"Up": False, "Right": True, "Down": True, "Left": True},
        2: {"Up": False, "Right": False, "Down": True, "Left": True},
        3: {"Up": True, "Right": True, "Down": True, "Left": False},
        4: {"Up": True, "Right": True, "Down": True, "Left": True},
        5: {"Up": True, "Right": False, "Down": True, "Left": True},
        6: {"Up": True, "Right": True, "Down": False, "Left": False},
        7: {"Up": True, "Right": True, "Down": False, "Left": True},
        8: {"Up": True, "Right": False, "Down": False, "Left": True}
    }
    return possibleActionsForZeroPosition[index]

#returns the index of the empty tile ("zero" tile) in a state
def findIndexOfZeroInState(state):
    index = 0
    for position in state:
        if position == 0:
            break
        index += 1
    return index

#effectively generates the children nodes for a state
def generateNextPossibleMovesForState(state):
    possibleActions = identifyActionsForState(state)
    index = findIndexOfZeroInState(state)
    nextStates = []
    #for each action available at the current state, add that action and its resulting state to a list to be returned
    if (possibleActions["Up"]):
        stateAfterMoveUp = list(state[:])
        stateAfterMoveUp[index],stateAfterMoveUp[index-3]=stateAfterMoveUp[index-3], stateAfterMoveUp[index]
        nextStates.append({"action": "Up","state": tuple(stateAfterMoveUp)})
    if (possibleActions["Right"]):
        stateAfterMoveRight = list(state[:])
        stateAfterMoveRight[index],stateAfterMoveRight[index+1]=stateAfterMoveRight[index+1], stateAfterMoveRight[index]
        nextStates.append({"action": "Right","state": tuple(stateAfterMoveRight)})
    if (possibleActions["Down"]):
        stateAfterMoveDown = list(state[:])
        stateAfterMoveDown[index],stateAfterMoveDown[index+3]=stateAfterMoveDown[index+3], stateAfterMoveDown[index]
        nextStates.append({"action": "Down","state": tuple(stateAfterMoveDown)})
    if (possibleActions["Left"]):
        stateAfterMoveLeft = list(state[:])
        stateAfterMoveLeft[index],stateAfterMoveLeft[index-1]=stateAfterMoveLeft[index-1], stateAfterMoveLeft[index]
        nextStates.append({"action": "Left","state": tuple(stateAfterMoveLeft)})
    return nextStates

#create a hash key for a state
#this honestly probably isn't even necessary, because tuples themselves can be used as a hash key in
#a dict since they are immutable, and states' usage is unique in the frontier and explored set,
#but I did it already and don't feel like refactoring, so nyeh
def generateStateHash(state):
    hash = 0
    power = 0
    for position in state:
        hash += position * pow(10,power)
        power += 1
    return hash

#Takes a node and traces the path (in reverse) up to the initial (puzzle) state
#Used to trace the optimal solution or just a list of ancestors (useful for IDS repeated state checking)
def traceSolution(node):
    optimalSolution = []
    currentNode = node
    #trace path to initial state by pushing nodes onto a stack and tracing by parent nodes, starting at node being passed
    while currentNode != None:
        optimalSolution.append(currentNode)
        currentNode = currentNode.parent
    return optimalSolution

#nicely prints out the contents of a Node object... should have refactored and put this in the
#node class, but I again don't feel like refactoring, sorryyyyy
def printNode(node):
    print(node.parent,node.state,node.action)
    return

In [16]:
###
#DATA STRUCTURES
###

#node structure used for maintaining the explored portion of the transition model
class Node:
    def __init__(self, parent, state, action):
        self.parent = parent #parent Node
        self.state = state #state, as a tuple
        self.action = action
        
#subclass of node that tracks the depth of each node, used for DFS in IDS
class DepthNode(Node):
    def __init__(self, parent, state, action, depth):
        Node.__init__(self,parent,state,action)
        self.depth = depth
    
#frontier structure used for maintaining the frontier in a more performant way than just a list used as a FIFO
#maintains a list that is used for push(LIFO push, FIFO enqueue)/pop (LIFO)/dequeue (FIFO) operations
#maintains a dict that is used for lookup operations
class Frontier:
    def __init__(self):
        self.list = []
        self.dict = dict()
        
    #LIFO push, FIFO enqueue
    def push(self,node):
        self.list.append(node)
        self.dict[generateStateHash(node.state)] = node
        return
    
    #FIFO dequeue
    def dequeue(self):
        poppedNode = self.list.pop(0)
        del self.dict[generateStateHash(poppedNode.state)]
        return poppedNode
    
    #LIFO pop
    def pop(self):
        poppedNode = self.list.pop()
        del self.dict[generateStateHash(poppedNode.state)]
        return poppedNode
    
    def isStateInFrontier(self,state):
        return generateStateHash(state) in self.dict
    
    def isEmpty(self):
        return len(self.list) == 0

In [17]:
###
#implementation of function "Iterative_deepening_DFS"
###
#takes a list or tuple representation of a solvable 8-puzzle state and returns a dictionary with the following elements:
#"solution": a stack (LIFO) of nodes with the initial state node on top and the goal state node on the bottom
#"nodesExplored": the total number of nodes explored during this execution of the IDS for the given initialState
#"time": time taken to execute the IDS for the given initialState

def Iterative_deepening_DFS(initialState):
    depth = 0
    nodesExplored = 0
    beginningTime = time.time()
    while True: #iteratively perform depth-limited DFS
        dlsSummary = depthLimitedSearch(initialState,depth)
        nodesExplored += dlsSummary["nodesExplored"]
        if dlsSummary["resultCandidate"].state == goalState:
            break
        depth += 1
    timeElapsedInSeconds = time.time() - beginningTime #track time
    optimalSolution = traceSolution(dlsSummary["resultCandidate"])
    return {"solution":optimalSolution,"nodesExplored":nodesExplored,"time":timeElapsedInSeconds}

def depthLimitedSearch(initialState, maxDepth):
    nodesExploredThisSearch = 0
    frontier = [] #frontier is LIFO stack
    frontier.append(DepthNode(None,tuple(initialState),None,0))
    while frontier:
        currentNode = frontier.pop()
        if currentNode.depth < maxDepth: #depth limiting logic
            moves = generateNextPossibleMovesForState(currentNode.state) #spawn children
            nodesExploredThisSearch += 1
            for move in moves:
                ancestors = set() #trace ancestors for backtrack checking
                for node in traceSolution(currentNode):
                    ancestors.add(node.state)
                if move["state"] not in ancestors: #check to make sure that this move will not backtrack the steps taken to get here
                    frontier.append(DepthNode(currentNode,move["state"],move["action"],currentNode.depth+1))
        if currentNode.state == goalState:
            break
    return {"resultCandidate":currentNode,"nodesExplored":nodesExploredThisSearch}

In [18]:
###
#implementation of function "breadthFirstSearch"
###
#takes a list or tuple representation of a solvable 8-puzzle state and returns a dictionary with the following elements:
#"solution": a stack (LIFO) of nodes with the initial state node on top and the goal state node on the bottom
#"nodesExplored": the total number of nodes explored during this execution of the BFS for the given initialState
#"time": time taken to execute the BFS for the given initialState


def breadthFirstSearch(initialState):
    #initialize search
    frontier = Frontier()
    frontier.push(Node(None,tuple(initialState), None)) #push == enqueue
    exploredStates = dict()
    currentNode = None; nodesExplored = 0
    beginningTime = time.time() #time the solution
    while True:
        #get a node to explore, figure out the next possible moves, and mark it as explored
        currentNode = frontier.dequeue()
        moves = generateNextPossibleMovesForState(currentNode.state) #spawn children
        exploredStates[generateStateHash(currentNode.state)] = currentNode.state; nodesExplored += 1
        for move in moves:
            if (not frontier.isStateInFrontier(move["state"])) and (generateStateHash(move["state"]) not in exploredStates):
                newNode = Node(currentNode,move["state"],move["action"])
                frontier.push(newNode) #enqueue
        if currentNode.state == goalState:
            break
    timeElapsedInSeconds = time.time() - beginningTime #track time
    optimalSolution = traceSolution(currentNode)
    #return optimal solution with metrics
    return {"solution":optimalSolution,"nodesExplored":nodesExplored,"time":timeElapsedInSeconds}

In [19]:
###
#implementation of function "print_result(result)"
###
#Directly takes the output of a breadthFirstSearch(initialState) or Iterative_deepening_DFS(initialState)
#and creates a nice output showing the step by step solution for the initialState
#Example usage:
#print_result(breadthFirstSearch(1,2,0,3,4,5,6,7,8)))
#print_result(Iterative_deepening_DFS(puzzleCases[4]))

def print_result(result):
    solution = result["solution"]
    while solution:
        step = solution.pop()
        state = step.state; action = step.action
        if action != None:
            print(action, "to")
        printStateNicely(state)
    return
    
#helper function, prints a state as a grid
def printStateNicely(state):
    print(state[0],state[1],state[2])
    print(state[3],state[4],state[5])
    print(state[6],state[7],state[8])
    return

In [20]:
###
#TESTING + METRICS CODE + OUTPUT
###
#Simple script to run IDS and BFS on each puzzle case
#Outputs nodes explored, time taken, and steps for each example puzzle
#After running through all 100 of the example puzzles, 
#NOTE: This does NOT call print_result(result)
#NOTE: This took about an hour to run on an i7-8700k processor @3.7GHz. The summary has been included in a markdown cell below.

#setup
puzzleIndex = 0
idsTime = 0
bfsTime = 0
idsSteps = []
bfsSteps = []
idsExplored = 0
bfsExplored = 0
for puzzle in puzzleCases:
    #DFS solution + metrics + puzzle output
    puzzleSolution = Iterative_deepening_DFS(puzzle)
    idsTime += puzzleSolution["time"]
    idsSteps.append(len(puzzleSolution["solution"]) - 1) #there are one fewer steps in the solution than there are elements in the solution
    idsExplored += puzzleSolution["nodesExplored"]
    print(puzzleIndex,"IDS: exploredNodes:",puzzleSolution["nodesExplored"],"time:", puzzleSolution["time"],"steps:", len(puzzleSolution["solution"]) - 1)
    #BFS solution + metrics + puzzle output
    puzzleSolution = breadthFirstSearch(puzzle)
    bfsTime += puzzleSolution["time"]
    bfsSteps.append(len(puzzleSolution["solution"]) - 1)
    bfsExplored += puzzleSolution["nodesExplored"]
    print(puzzleIndex,"BFS: exploredNodes:",puzzleSolution["nodesExplored"],"time:", puzzleSolution["time"],"steps:", len(puzzleSolution["solution"]) - 1)
    #iterate index
    puzzleIndex += 1

#summary metrics
averageIDSTime = idsTime/(puzzleIndex + 1.0)
averageBFSTime = bfsTime/(puzzleIndex + 1.0)
totalIDSSteps = 0
for stepsCount in idsSteps:
    totalIDSSteps += stepsCount
totalBFSSteps = 0
for stepsCount in bfsSteps:
    totalBFSSteps += stepsCount
averageIDSSteps = (totalIDSSteps + 0.0)/(puzzleIndex + 1.0)
averageBFSSteps = (totalBFSSteps + 0.0)/(puzzleIndex + 1.0)
averageIDSExplored = idsExplored/(puzzleIndex + 1.0)
averageBFSExplored = bfsExplored/(puzzleIndex + 1.0)

#print summary for IDS and BFS over 100 puzzle cases
print("IDS: ","Average Steps:",averageIDSSteps,"Average Time:",averageIDSTime, "Average Nodes Explored:",averageIDSExplored)
print("BFS: ","Average Steps:",averageBFSSteps,"Average Time:",averageBFSTime, "Average Nodes Explored:",averageBFSExplored)

0 IDS: exploredNodes: 4593089 time: 74.0620629787445 steps: 25
0 BFS: exploredNodes: 148677 time: 5.196113586425781 steps: 25
1 IDS: exploredNodes: 4355760 time: 69.58101963996887 steps: 25
1 BFS: exploredNodes: 152077 time: 5.228026866912842 steps: 25
2 IDS: exploredNodes: 1093447 time: 15.962395191192627 steps: 23
2 BFS: exploredNodes: 120981 time: 4.16686224937439 steps: 23
3 IDS: exploredNodes: 3049995 time: 47.61273217201233 steps: 25
3 BFS: exploredNodes: 157107 time: 5.3946075439453125 steps: 25
4 IDS: exploredNodes: 30442 time: 0.34108877182006836 steps: 16
4 BFS: exploredNodes: 12995 time: 0.4408280849456787 steps: 16
5 IDS: exploredNodes: 504324 time: 7.043183088302612 steps: 21
5 BFS: exploredNodes: 63820 time: 2.2350077629089355 steps: 21
6 IDS: exploredNodes: 15110045 time: 258.4870836734772 steps: 28
6 BFS: exploredNodes: 177731 time: 5.970041036605835 steps: 28
7 IDS: exploredNodes: 489792 time: 6.815792083740234 steps: 21
7 BFS: exploredNodes: 65896 time: 2.311815977096

64 IDS: exploredNodes: 1501561 time: 22.329322338104248 steps: 24
64 BFS: exploredNodes: 139643 time: 4.752259016036987 steps: 24
65 IDS: exploredNodes: 210137 time: 2.740703582763672 steps: 20
65 BFS: exploredNodes: 42590 time: 1.4730627536773682 steps: 20
66 IDS: exploredNodes: 15138592 time: 256.58413577079773 steps: 28
66 BFS: exploredNodes: 179456 time: 6.024896860122681 steps: 28
67 IDS: exploredNodes: 1848367 time: 28.23755145072937 steps: 24
67 BFS: exploredNodes: 134109 time: 4.592694282531738 steps: 24
68 IDS: exploredNodes: 15732961 time: 269.1106791496277 steps: 28
68 BFS: exploredNodes: 178656 time: 6.040853500366211 steps: 28
69 IDS: exploredNodes: 9264642 time: 152.75075483322144 steps: 27
69 BFS: exploredNodes: 176807 time: 6.007966995239258 steps: 27
70 IDS: exploredNodes: 546487 time: 7.715377330780029 steps: 22
70 BFS: exploredNodes: 75048 time: 2.6319732666015625 steps: 22
71 IDS: exploredNodes: 312388 time: 4.207750082015991 steps: 20
71 BFS: exploredNodes: 53844 t

# IDS and BFS performance per puzzle + summary

## This test was performed on an i7-8700k CPU @3.7GHz and took approximately 65 minutes to complete.
`
0 IDS: exploredNodes: 4593089 time: 74.0620629787445 steps: 25
0 BFS: exploredNodes: 148677 time: 5.196113586425781 steps: 25
1 IDS: exploredNodes: 4355760 time: 69.58101963996887 steps: 25
1 BFS: exploredNodes: 152077 time: 5.228026866912842 steps: 25
2 IDS: exploredNodes: 1093447 time: 15.962395191192627 steps: 23
2 BFS: exploredNodes: 120981 time: 4.16686224937439 steps: 23
3 IDS: exploredNodes: 3049995 time: 47.61273217201233 steps: 25
3 BFS: exploredNodes: 157107 time: 5.3946075439453125 steps: 25
4 IDS: exploredNodes: 30442 time: 0.34108877182006836 steps: 16
4 BFS: exploredNodes: 12995 time: 0.4408280849456787 steps: 16
5 IDS: exploredNodes: 504324 time: 7.043183088302612 steps: 21
5 BFS: exploredNodes: 63820 time: 2.2350077629089355 steps: 21
6 IDS: exploredNodes: 15110045 time: 258.4870836734772 steps: 28
6 BFS: exploredNodes: 177731 time: 5.970041036605835 steps: 28
7 IDS: exploredNodes: 489792 time: 6.815792083740234 steps: 21
7 BFS: exploredNodes: 65896 time: 2.3118159770965576 steps: 21
8 IDS: exploredNodes: 248117 time: 3.321131706237793 steps: 20
8 BFS: exploredNodes: 43695 time: 1.5248939990997314 steps: 20
9 IDS: exploredNodes: 197444 time: 2.582099199295044 steps: 20
9 BFS: exploredNodes: 46309 time: 1.5987553596496582 steps: 20
10 IDS: exploredNodes: 1544313 time: 22.981597423553467 steps: 24
10 BFS: exploredNodes: 119663 time: 4.167861700057983 steps: 24
11 IDS: exploredNodes: 133169 time: 1.6665418148040771 steps: 19
11 BFS: exploredNodes: 39242 time: 1.346388816833496 steps: 19
12 IDS: exploredNodes: 455142 time: 6.276202440261841 steps: 21
12 BFS: exploredNodes: 68570 time: 2.358713150024414 steps: 21
13 IDS: exploredNodes: 452002 time: 6.222364902496338 steps: 21
13 BFS: exploredNodes: 68957 time: 2.3726422786712646 steps: 21
14 IDS: exploredNodes: 4470072 time: 70.82768416404724 steps: 25
14 BFS: exploredNodes: 150555 time: 5.19810938835144 steps: 25
15 IDS: exploredNodes: 3301334 time: 51.22809815406799 steps: 25
15 BFS: exploredNodes: 159093 time: 5.375603437423706 steps: 25
16 IDS: exploredNodes: 90923 time: 1.112056016921997 steps: 18
16 BFS: exploredNodes: 18280 time: 0.6313207149505615 steps: 18
17 IDS: exploredNodes: 58209 time: 0.6841697692871094 steps: 18
17 BFS: exploredNodes: 26177 time: 0.8766589164733887 steps: 18
18 IDS: exploredNodes: 4760770 time: 75.95098495483398 steps: 25
18 BFS: exploredNodes: 146333 time: 5.031548261642456 steps: 25
19 IDS: exploredNodes: 243087 time: 3.2453341484069824 steps: 20
19 BFS: exploredNodes: 38485 time: 1.3533799648284912 steps: 20
20 IDS: exploredNodes: 4491402 time: 71.12787318229675 steps: 26
20 BFS: exploredNodes: 163539 time: 5.542210817337036 steps: 26
21 IDS: exploredNodes: 1026070 time: 14.861247777938843 steps: 23
21 BFS: exploredNodes: 115260 time: 4.027263164520264 steps: 23
22 IDS: exploredNodes: 1642909 time: 24.565343618392944 steps: 24
22 BFS: exploredNodes: 131549 time: 4.528894901275635 steps: 24
23 IDS: exploredNodes: 1495959 time: 22.503852367401123 steps: 23
23 BFS: exploredNodes: 108232 time: 3.7669312953948975 steps: 23
24 IDS: exploredNodes: 3206507 time: 49.33812713623047 steps: 25
24 BFS: exploredNodes: 145770 time: 5.026564359664917 steps: 25
25 IDS: exploredNodes: 317307 time: 4.250637531280518 steps: 20
25 BFS: exploredNodes: 53116 time: 1.8380954265594482 steps: 20
26 IDS: exploredNodes: 357112 time: 4.75728964805603 steps: 21
26 BFS: exploredNodes: 76692 time: 2.617011785507202 steps: 21
27 IDS: exploredNodes: 178960 time: 2.301846742630005 steps: 19
27 BFS: exploredNodes: 30330 time: 1.057178020477295 steps: 19
28 IDS: exploredNodes: 14380764 time: 242.3930847644806 steps: 28
28 BFS: exploredNodes: 177777 time: 5.919150114059448 steps: 28
29 IDS: exploredNodes: 259827 time: 3.3799655437469482 steps: 20
29 BFS: exploredNodes: 59669 time: 2.04055118560791 steps: 20
30 IDS: exploredNodes: 5415496 time: 87.55798768997192 steps: 26
30 BFS: exploredNodes: 162409 time: 5.492312908172607 steps: 26
31 IDS: exploredNodes: 3277736 time: 51.265928983688354 steps: 25
31 BFS: exploredNodes: 148466 time: 5.125339508056641 steps: 25
32 IDS: exploredNodes: 181365 time: 2.3208022117614746 steps: 19
32 BFS: exploredNodes: 30001 time: 1.0372259616851807 steps: 19
33 IDS: exploredNodes: 1026813 time: 14.765536785125732 steps: 23
33 BFS: exploredNodes: 120257 time: 4.137934446334839 steps: 23
34 IDS: exploredNodes: 794081 time: 11.672805070877075 steps: 22
34 BFS: exploredNodes: 76993 time: 2.662879228591919 steps: 22
35 IDS: exploredNodes: 1761747 time: 26.596919775009155 steps: 24
35 BFS: exploredNodes: 135808 time: 4.632584571838379 steps: 24
36 IDS: exploredNodes: 396584 time: 5.393561601638794 steps: 21
36 BFS: exploredNodes: 64847 time: 2.2779388427734375 steps: 21
37 IDS: exploredNodes: 5245380 time: 83.82194662094116 steps: 26
37 BFS: exploredNodes: 161348 time: 5.463404655456543 steps: 26
38 IDS: exploredNodes: 549113 time: 7.722347021102905 steps: 21
38 BFS: exploredNodes: 58802 time: 2.0644819736480713 steps: 21
39 IDS: exploredNodes: 588262 time: 8.174157857894897 steps: 22
39 BFS: exploredNodes: 90719 time: 3.103703260421753 steps: 22
40 IDS: exploredNodes: 11085169 time: 184.51081013679504 steps: 27
40 BFS: exploredNodes: 174919 time: 5.884271621704102 steps: 27
41 IDS: exploredNodes: 1889777 time: 29.028412580490112 steps: 24
41 BFS: exploredNodes: 123286 time: 4.272571802139282 steps: 24
42 IDS: exploredNodes: 8952 time: 0.09375929832458496 steps: 14
42 BFS: exploredNodes: 3590 time: 0.12366390228271484 steps: 14
43 IDS: exploredNodes: 9117893 time: 149.6210744380951 steps: 27
43 BFS: exploredNodes: 177535 time: 5.939125061035156 steps: 27
44 IDS: exploredNodes: 1050895 time: 15.247245073318481 steps: 23
44 BFS: exploredNodes: 122022 time: 4.160904169082642 steps: 23
45 IDS: exploredNodes: 193529 time: 2.5222384929656982 steps: 20
45 BFS: exploredNodes: 51737 time: 1.7543296813964844 steps: 20
46 IDS: exploredNodes: 1550083 time: 23.152114868164062 steps: 23
46 BFS: exploredNodes: 105604 time: 3.658201217651367 steps: 23
47 IDS: exploredNodes: 236191 time: 3.1316182613372803 steps: 20
47 BFS: exploredNodes: 45448 time: 1.5608484745025635 steps: 20
48 IDS: exploredNodes: 155573 time: 1.9717340469360352 steps: 19
48 BFS: exploredNodes: 29169 time: 1.026259183883667 steps: 19
49 IDS: exploredNodes: 536215 time: 7.491971492767334 steps: 22
49 BFS: exploredNodes: 91499 time: 3.16554594039917 steps: 22
50 IDS: exploredNodes: 1594870 time: 23.946969747543335 steps: 24
50 BFS: exploredNodes: 120025 time: 4.1648876667022705 steps: 24
51 IDS: exploredNodes: 2092899 time: 31.16969084739685 steps: 24
51 BFS: exploredNodes: 147497 time: 4.969716548919678 steps: 24
52 IDS: exploredNodes: 615981 time: 8.739611625671387 steps: 22
52 BFS: exploredNodes: 74727 time: 2.6100542545318604 steps: 22
53 IDS: exploredNodes: 1342 time: 0.011975288391113281 steps: 11
53 BFS: exploredNodes: 1271 time: 0.042887210845947266 steps: 11
54 IDS: exploredNodes: 1034433 time: 15.170445680618286 steps: 23
54 BFS: exploredNodes: 117859 time: 4.065136194229126 steps: 23
55 IDS: exploredNodes: 58735 time: 0.6971437931060791 steps: 18
55 BFS: exploredNodes: 26180 time: 0.8905918598175049 steps: 18
56 IDS: exploredNodes: 378444 time: 5.1332690715789795 steps: 21
56 BFS: exploredNodes: 75675 time: 2.6140222549438477 steps: 21
57 IDS: exploredNodes: 416504 time: 5.673840284347534 steps: 21
57 BFS: exploredNodes: 72468 time: 2.5003890991210938 steps: 21
58 IDS: exploredNodes: 1164269 time: 17.000538110733032 steps: 23
58 BFS: exploredNodes: 119325 time: 4.128962516784668 steps: 23
59 IDS: exploredNodes: 158122 time: 2.0126471519470215 steps: 19
59 BFS: exploredNodes: 34835 time: 1.1938109397888184 steps: 19
60 IDS: exploredNodes: 1053878 time: 15.400832653045654 steps: 22
60 BFS: exploredNodes: 86717 time: 2.9979968070983887 steps: 22
61 IDS: exploredNodes: 9411587 time: 154.18285393714905 steps: 27
61 BFS: exploredNodes: 175521 time: 5.91023325920105 steps: 27
62 IDS: exploredNodes: 82409 time: 1.0053071975708008 steps: 18
62 BFS: exploredNodes: 20742 time: 0.7110965251922607 steps: 18
63 IDS: exploredNodes: 726206 time: 10.448079109191895 steps: 22
63 BFS: exploredNodes: 74682 time: 2.5720977783203125 steps: 22
64 IDS: exploredNodes: 1501561 time: 22.329322338104248 steps: 24
64 BFS: exploredNodes: 139643 time: 4.752259016036987 steps: 24
65 IDS: exploredNodes: 210137 time: 2.740703582763672 steps: 20
65 BFS: exploredNodes: 42590 time: 1.4730627536773682 steps: 20
66 IDS: exploredNodes: 15138592 time: 256.58413577079773 steps: 28
66 BFS: exploredNodes: 179456 time: 6.024896860122681 steps: 28
67 IDS: exploredNodes: 1848367 time: 28.23755145072937 steps: 24
67 BFS: exploredNodes: 134109 time: 4.592694282531738 steps: 24
68 IDS: exploredNodes: 15732961 time: 269.1106791496277 steps: 28
68 BFS: exploredNodes: 178656 time: 6.040853500366211 steps: 28
69 IDS: exploredNodes: 9264642 time: 152.75075483322144 steps: 27
69 BFS: exploredNodes: 176807 time: 6.007966995239258 steps: 27
70 IDS: exploredNodes: 546487 time: 7.715377330780029 steps: 22
70 BFS: exploredNodes: 75048 time: 2.6319732666015625 steps: 22
71 IDS: exploredNodes: 312388 time: 4.207750082015991 steps: 20
71 BFS: exploredNodes: 53844 time: 1.8779783248901367 steps: 20
72 IDS: exploredNodes: 2293012 time: 35.54197430610657 steps: 24
72 BFS: exploredNodes: 123939 time: 4.270585060119629 steps: 24
73 IDS: exploredNodes: 1733644 time: 26.34261441230774 steps: 24
73 BFS: exploredNodes: 125054 time: 4.359344959259033 steps: 24
74 IDS: exploredNodes: 3534544 time: 56.404245138168335 steps: 25
74 BFS: exploredNodes: 149879 time: 5.218050003051758 steps: 25
75 IDS: exploredNodes: 1805509 time: 27.80963921546936 steps: 24
75 BFS: exploredNodes: 116298 time: 3.978365182876587 steps: 24
76 IDS: exploredNodes: 1974477 time: 31.58160972595215 steps: 24
76 BFS: exploredNodes: 131947 time: 4.577739000320435 steps: 24
77 IDS: exploredNodes: 541143 time: 7.654546022415161 steps: 21
77 BFS: exploredNodes: 59313 time: 2.1123154163360596 steps: 21
78 IDS: exploredNodes: 175195 time: 2.2400381565093994 steps: 19
78 BFS: exploredNodes: 29947 time: 1.0372278690338135 steps: 19
79 IDS: exploredNodes: 3886593 time: 61.75892734527588 steps: 25
79 BFS: exploredNodes: 148796 time: 5.121284246444702 steps: 25
80 IDS: exploredNodes: 398362 time: 5.511268377304077 steps: 21
80 BFS: exploredNodes: 73977 time: 2.6080563068389893 steps: 21
81 IDS: exploredNodes: 381357 time: 5.200107574462891 steps: 21
81 BFS: exploredNodes: 62395 time: 2.169191360473633 steps: 21
82 IDS: exploredNodes: 784824 time: 11.295804262161255 steps: 22
82 BFS: exploredNodes: 97740 time: 3.421847343444824 steps: 22
83 IDS: exploredNodes: 1015497 time: 14.89416790008545 steps: 23
83 BFS: exploredNodes: 122993 time: 4.3015289306640625 steps: 23
84 IDS: exploredNodes: 720005 time: 10.529853105545044 steps: 22
84 BFS: exploredNodes: 82810 time: 2.923187494277954 steps: 22
85 IDS: exploredNodes: 1506902 time: 22.775123596191406 steps: 23
85 BFS: exploredNodes: 107570 time: 3.799868583679199 steps: 23
86 IDS: exploredNodes: 124953 time: 1.5618276596069336 steps: 19
86 BFS: exploredNodes: 40325 time: 1.3793120384216309 steps: 19
87 IDS: exploredNodes: 2587654 time: 40.325188636779785 steps: 24
87 BFS: exploredNodes: 142456 time: 5.008613348007202 steps: 24
88 IDS: exploredNodes: 1351553 time: 20.24790644645691 steps: 23
88 BFS: exploredNodes: 114064 time: 3.9893431663513184 steps: 23
89 IDS: exploredNodes: 3208725 time: 50.530911445617676 steps: 25
89 BFS: exploredNodes: 160071 time: 5.477359056472778 steps: 25
90 IDS: exploredNodes: 3318371 time: 52.52463126182556 steps: 25
90 BFS: exploredNodes: 147298 time: 5.053492069244385 steps: 25
91 IDS: exploredNodes: 1116199 time: 16.414119720458984 steps: 23
91 BFS: exploredNodes: 120804 time: 4.184806823730469 steps: 23
92 IDS: exploredNodes: 1119047 time: 16.58367085456848 steps: 23
92 BFS: exploredNodes: 120591 time: 4.1628758907318115 steps: 23
93 IDS: exploredNodes: 83844 time: 1.0252599716186523 steps: 18
93 BFS: exploredNodes: 19306 time: 0.6692168712615967 steps: 18
94 IDS: exploredNodes: 59297 time: 0.7021040916442871 steps: 18
94 BFS: exploredNodes: 26083 time: 0.8816149234771729 steps: 18
95 IDS: exploredNodes: 487345 time: 6.721034049987793 steps: 21
95 BFS: exploredNodes: 65623 time: 2.293896436691284 steps: 21
96 IDS: exploredNodes: 158269 time: 2.010599374771118 steps: 19
96 BFS: exploredNodes: 34718 time: 1.1998190879821777 steps: 19
97 IDS: exploredNodes: 364674 time: 4.903864622116089 steps: 21
97 BFS: exploredNodes: 76451 time: 2.6179685592651367 steps: 21
98 IDS: exploredNodes: 1589804 time: 23.984890460968018 steps: 24
98 BFS: exploredNodes: 134781 time: 4.611701488494873 steps: 24
99 IDS: exploredNodes: 10680576 time: 181.60154795646667 steps: 27
99 BFS: exploredNodes: 175058 time: 6.145601749420166 steps: 27
IDS:  Average Steps: 22.10891089108911 Average Time: 36.18309720435945 Average Nodes Explored: 2256884.5148514854
BFS:  Average Steps: 22.10891089108911 Average Time: 3.3369736411783957 Average Nodes Explored: 97019.80198019803
`

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
===============================

In [ ]:
#you need to implement print_result function to print out the result according to the required format
print_result(result)


# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                Average_Steps    Average_Time      
 IDS
 
 BFS

